<a href="https://colab.research.google.com/github/slowandfast/DiagRecommdSystem_01/blob/main/KJH_BERT_EMR%EA%B8%B0%EB%B0%98_%EC%A7%84%EB%8B%A8%EC%B6%94%EC%B2%9C%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
file_path = '/content/신장내과_본원_외래_20230701_20231231_filtered.xlsx'

selected_columns = ['ICD10', 'transstr']
df = pd.read_excel(file_path, usecols=selected_columns)

print('전체 데이터 수 :',len(df))
df.head(5)

전체 데이터 수 : 826


,ICD10,transstr
0,N391,115-61-61 2023.5.10.Suspicion of proteinuria i...
1,N184,"130/61/69 Symptoms (-), edema (-), urbanic sym..."
2,N184,"153/75/85 osophy (-), edema (-), urinary sympt..."
3,N185,148/90/64 Idema Mild CKD D/T ADPKD has been tr...
4,N183,"Hello teacher, I am a patient who is being hos..."


In [4]:
# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_temp, Y_train, Y_temp = train_test_split(df['transstr'], df['ICD10'], test_size=0.2, random_state=5)
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=7)

print(X_train)
print("=======================================\n")
print(X_test)
print("=======================================\n")
print(X_val)
print("=======================================\n")
print(Y_train)
print("=======================================\n")
print(Y_test)
print("=======================================\n")
print(Y_val)

443    2021.09 CABG surgery 2023.06 CR 4.0, EGFR 14 2...
441    Since 2016, Hematuria had hematuria in 2022 TY...
66     23.04 LMC obstetrics and gynecology visits Hem...
773    60s HTN Diagnosis 2023.11.21 CR 1.58 BP 121/67...
131    I didn't have a previous checkup.2023. CR 1.83...
                             ...                        
73     156/94/83, HOME BP <130/80, and Type A told fo...
400    # HTN, On Medication.Comprehensive Award at th...
118    The patient is a patient of 72 years old, curr...
701    HTN, DM 25 -year LMC Drug During LMC Pharmaceu...
206    The patient is a 69 -year -old (m) patient cur...
Name: transstr, Length: 660, dtype: object

278    114-69-88 2 years ago Urology and Browno Holy ...
690    97-53-112 Diabetes and Hypertension-more than ...
108    145-75-85 2023.4.Microscopic Hematuria 2022.10...
499    2023/04 I heard that there is proteinuria duri...
755    The patient is a patient of 77 years old, curr...
                             ...            

In [6]:
#https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [5]:
# device_name = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device(device_name)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('NitzanBar/umls-spanbert')

# Model
model = AutoModelForSequenceClassification.from_pretrained('NitzanBar/umls-spanbert')
# model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [114]:
class EMRDiagcdDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        record = self.df.iloc[index]['transstr']
        # diagcd = self.df.iloc[index]['ICD10']
        diagcd = self.df.iloc[index]['encoded_ICD10']

        return record, diagcd

        # tokens = self.tokenizer(
        #     record,
        #     return_tensors='pt',
        #     truncation=True,
        #     padding='max_length')

        # input_ids = tokens[input_ids].squeeze(0)
        # attention_mask  = tokens[attention_mask ].squeeze(0)
        # token_type_ids  = torch.zeros_like(token_type_ids )

        # return {
        #     'input_ids' : input_ids,
        #     'attention_mask' : attention_mask,
        #     'token_type_ids' : token_type_ids,
        # }, torch.tensor(diagcd)

In [115]:
# label encoding
unique_label = Y_train.unique()
print("1 :", unique_label)

label_encoder = LabelEncoder()
label_encoder.fit(unique_label)

# print("2 : ", label_encoder)

temp_df = Y_train.to_frame()
temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10'])
Y_train_encoding = pd.Series(temp_df['encoded_ICD10'])
# Y_train_encoding = Y_train_encoding.rename('encoded_ICD10')
# print("2 :", Y_train_encoding)

1 : ['N185' 'N183' 'R311' 'R601' 'N184' 'N178' 'N391' 'N182' 'N028' 'Z524'
 'R808' 'N029' 'N059' 'R798']


In [104]:
print(Y_train)

443    N185
441    N183
66     R311
773    N183
131    N183
       ... 
73     Z524
400    R311
118    N183
701    N184
206    N183
Name: ICD10, Length: 660, dtype: object


In [91]:
print(Y_train_encoding[:5])

[7 5 9 5 5]


In [116]:
train_df = pd.concat([X_train, Y_train_encoding, Y_train], axis=1)
train_dataset = EMRDiagcdDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [109]:
train_df.to_excel('/content/train_df.xlsx')

In [117]:
optimizer = Adam(model.parameters(), lr=1e-2)
criterion = torch.nn.Softmax()
itr = 1
p_itr = 50
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [ ]:
# train
model.train()

for epoch in range(epochs):
    for record, diagcd in train_loader:
        optimizer.zero_grad()
        # print("record: ", record)
        # print("diagcd: ", diagcd)
        encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
        # encoded_record = encoded_record.to(device)
        # print(type(encoded_record))
        diagcd = torch.tensor(diagcd)
        # diagcd = torch.tensor(list(diagcd), dim=0).to(device)

        outputs = model(**encoded_record, labels=diagcd)

        loss = outputs.loss
        logits = outputs.logits

        predict = torch.argmax(F.softmax(logits), dim=14)
        correct = predict.eq(diagcd)

        total_correct += correct.sum().item()
        total_len += len(diagcd)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr += 1

<ipython-input-118-0be101af0015>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  diagcd = torch.tensor(diagcd)


In [52]:
test_df = pd.concat([X_test, Y_test], axis=1)
print(test_df)

test_dataset = EMRDiagcdDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

                                              transstr ICD10
278  114-69-88 2 years ago Urology and Browno Holy ...  R311
690  97-53-112 Diabetes and Hypertension-more than ...  N183
108  145-75-85 2023.4.Microscopic Hematuria 2022.10...  R311
499  2023/04 I heard that there is proteinuria duri...  R311
755  The patient is a patient of 77 years old, curr...  N183
..                                                 ...   ...
228  The patient is a diagnostic name [Paralysis of...  N183
151  # HTN (2022.)2023. CR 1.24 EGFR 41 EGFR K 5.7 ...  N183
129  The patient is a diagnostic [DJD] patient of 8...  N183
781  We will ask for consultation.The patient is a ...  R311
159  20 years ago, DM, HTN diagnosis 10 years ago, ...  N183

[83 rows x 2 columns]


In [ ]:
# evaluation
model.eval()

total_len = 0
total_correct = 0

for record, diagcd in test_loader:

    encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
    encoded_record, diagcd = encoded_record.to(device), diagcd.to(device)

    outputs = model(**encoded_record, labels=diagcd)

    logits = outputs.logits

    predict = torch.argmax(F.softmax(logits), dim=14)
    correct = predict.eq(diagcd)

    total_correct += correct.sum().item()
    total_len += len(diagcd)

print('Test accuracy: ', total_correct / total_len)